In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn import cluster
from scipy import stats  ## for z-score. do we still need it?

In [2]:
# Read speeds
speeds = pd.read_csv('speeds_pivoted.csv',sep=",")

speeds.head()

,speed_weekly_hrs,Number of Records,Pivot Field Names,Pivot Field Values,Lat,Location Index,Lon
0,81.113333,1,100,81.113333,-38.100617,0,145.488326
1,66.043333,1,100,66.043333,-38.062604,1,145.367474
2,57.036667,1,100,57.036667,-38.357822,2,144.757135
3,73.140000,1,100,73.140000,-38.355186,3,144.752776
4,62.690000,1,100,62.690000,-38.351950,4,144.918912


In [3]:
speeds.size

6212640

In [4]:
speeds = speeds.drop(['Pivot Field Values', 'Number of Records'], axis=1)

In [5]:
speeds = speeds.rename(columns={'Pivot Field Names': 'weekly_hour', 'speed_weekly_hrs': 'mean_speed'})

In [6]:
speeds.head()

,mean_speed,weekly_hour,Lat,Location Index,Lon
0,81.113333,100,-38.100617,0,145.488326
1,66.043333,100,-38.062604,1,145.367474
2,57.036667,100,-38.357822,2,144.757135
3,73.140000,100,-38.355186,3,144.752776
4,62.690000,100,-38.351950,4,144.918912


In [7]:
# Read crash
crashes_ = pd.read_csv('Crashes_Last_Five_Years.csv',sep="	")

crashes_.head()

,X,Y,OBJECTID,ACCIDENT_NO,ABS_CODE,ACCIDENT_STATUS,ACCIDENT_DATE,ACCIDENT_TIME,ALCOHOLTIME,ACCIDENT_TYPE,...,DEG_URBAN_ALL,LGA_NAME_ALL,REGION_NAME_ALL,SRNS,SRNS_ALL,RMA,RMA_ALL,DIVIDED,DIVIDED_ALL,STAT_DIV_NAME
0,145.060690,-37.810373,2693452,T20120013207,ABS to receive accident,Finished,26/5/2012,11.40.00,No,Collision with vehicle,...,Metropolitan Excluding CBD,BOROONDARA,METROPOLITAN SOUTH EAST REGION,NaN,NaN,Arterial Other,Arterial Other,Undivided,Undivided,Metro
1,144.991172,-37.883157,2693453,T20120013209,ABS to receive accident,Finished,6/6/2012,15.40.00,No,Collision with vehicle,...,Metropolitan Excluding CBD,PORT PHILLIP,METROPOLITAN NORTH WEST REGION,NaN,NaN,Arterial Other,"Arterial Other,Local Road",Undivided,Undivided,Metro
2,145.009458,-37.826952,2693454,T20120013212,ABS to receive accident,Finished,24/5/2012,09.00.00,No,Collision with vehicle,...,Metropolitan Excluding CBD,YARRA,METROPOLITAN NORTH WEST REGION,NaN,NaN,Arterial Other,"Arterial Other,Local Road",Undivided,Undivided,Metro
3,145.134597,-37.841546,2693455,T20120013213,ABS to receive accident,Finished,6/6/2012,17.02.00,No,Struck Pedestrian,...,Metropolitan Excluding CBD,WHITEHORSE,METROPOLITAN SOUTH EAST REGION,NaN,NaN,Local Road,Local Road,Undivided,Undivided,Metro
4,145.294600,-37.888597,2693456,T20120013219,ABS to receive accident,Finished,22/5/2012,16.30.00,No,Collision with vehicle,...,Metropolitan Excluding CBD,KNOX,METROPOLITAN SOUTH EAST REGION,NaN,NaN,Arterial Highway,"Arterial Highway,Local Road,Arterial Other",Undivided,Undivided,Metro


In [8]:
list(crashes_)

['X',
 'Y',
 'OBJECTID',
 'ACCIDENT_NO',
 'ABS_CODE',
 'ACCIDENT_STATUS',
 'ACCIDENT_DATE',
 'ACCIDENT_TIME',
 'ALCOHOLTIME',
 'ACCIDENT_TYPE',
 'DAY_OF_WEEK',
 'DCA_CODE',
 'HIT_RUN_FLAG',
 'LIGHT_CONDITION',
 'POLICE_ATTEND',
 'ROAD_GEOMETRY',
 'SEVERITY',
 'SPEED_ZONE',
 'RUN_OFFROAD',
 'NODE_ID',
 'LONGITUDE',
 'LATITUDE',
 'NODE_TYPE',
 'LGA_NAME',
 'REGION_NAME',
 'VICGRID_X',
 'VICGRID_Y',
 'TOTAL_PERSONS',
 'INJ_OR_FATAL',
 'FATALITY',
 'SERIOUSINJURY',
 'OTHERINJURY',
 'NONINJURED',
 'MALES',
 'FEMALES',
 'BICYCLIST',
 'PASSENGER',
 'DRIVER',
 'PEDESTRIAN',
 'PILLION',
 'MOTORIST',
 'UNKNOWN',
 'PED_CYCLIST_5_12',
 'PED_CYCLIST_13_18',
 'OLD_PEDESTRIAN',
 'OLD_DRIVER',
 'YOUNG_DRIVER',
 'ALCOHOL_RELATED',
 'UNLICENCSED',
 'NO_OF_VEHICLES',
 'HEAVYVEHICLE',
 'PASSENGERVEHICLE',
 'MOTORCYCLE',
 'PUBLICVEHICLE',
 'DEG_URBAN_NAME',
 'DEG_URBAN_ALL',
 'LGA_NAME_ALL',
 'REGION_NAME_ALL',
 'SRNS',
 'SRNS_ALL',
 'RMA',
 'RMA_ALL',
 'DIVIDED',
 'DIVIDED_ALL',
 'STAT_DIV_NAME']

In [9]:
speeds = speeds.rename(columns={'Lon': 'LONGITUDE', 'Lat': 'LATITUDE'})

In [10]:
#speeds_crashes = speeds.append(crashes, ignore_index=True)
#speeds_crashes = pd.concat([speeds, crashes], axis=1, sort=False)
#speeds_crashes = pd.merge(speeds, crashes, how='outer', on=['Lon', 'Lat'])

In [11]:
crashes_ = crashes_.drop(['X', 'Y', 'OBJECTID', 'ACCIDENT_NO', 'ABS_CODE', 'ACCIDENT_STATUS', 'DCA_CODE', 'NODE_ID','VICGRID_X', 
                        'LGA_NAME_ALL', 'REGION_NAME_ALL', 'RMA', 'RMA_ALL', 'DIVIDED', 'DIVIDED_ALL', 'STAT_DIV_NAME', 'VICGRID_Y', 'DEG_URBAN_NAME','DEG_URBAN_ALL'], axis=1)
crashes = crashes_['']

del(crashes_)

crashes.dtypes

ACCIDENT_DATE         object
ACCIDENT_TIME         object
ALCOHOLTIME           object
ACCIDENT_TYPE         object
DAY_OF_WEEK           object
HIT_RUN_FLAG          object
LIGHT_CONDITION       object
POLICE_ATTEND         object
ROAD_GEOMETRY         object
SEVERITY              object
SPEED_ZONE            object
RUN_OFFROAD           object
LONGITUDE            float64
LATITUDE             float64
NODE_TYPE             object
LGA_NAME              object
REGION_NAME           object
TOTAL_PERSONS          int64
INJ_OR_FATAL           int64
FATALITY               int64
SERIOUSINJURY          int64
OTHERINJURY            int64
NONINJURED             int64
MALES                  int64
FEMALES                int64
BICYCLIST              int64
PASSENGER              int64
DRIVER                 int64
PEDESTRIAN             int64
PILLION                int64
MOTORIST               int64
UNKNOWN                int64
PED_CYCLIST_5_12       int64
PED_CYCLIST_13_18      int64
OLD_PEDESTRIAN

In [12]:
#Do a cross join
speeds['tmp'] = 1
crashes['tmp'] = 1

speeds_crashes = pd.merge(speeds, crashes, on=['tmp'])
speeds_crashes = speeds_crashes.drop('tmp', axis=1)

#save memory

speeds_key = speeds['tmp']
speeds_crashes = pd.DataFrame(columns=(crashes.columns.append(speeds.columns)).unique())

MemoryError: 

In [ ]:
######
df1 = pd.read_csv("yourdata.csv")
df2 = pd.read_csv("yourdata2.csv")
df2_key = df2.Colname2

# creating a empty bucket to save result
df_result = pd.DataFrame(columns=(df1.columns.append(df2.columns)).unique())
df_result.to_csv("df3.csv",index_label=False)

# save data which only appear in df1 # sorry I was doing left join here. no need to run below two line.
# df_result = df1[df1.Colname1.isin(df2.Colname2)!=True]
# df_result.to_csv("df3.csv",index_label=False, mode="a")

# deleting df2 to save memory
del(df2)

def preprocess(x):
    df2=pd.merge(df1,x, left_on = "Colname1", right_on = "Colname2")
    df2.to_csv("df3.csv",mode="a",header=False,index=False)

reader = pd.read_csv("yourdata2.csv", chunksize=1000) # chunksize depends with you colsize

[preprocess(r) for r in reader]

In [ ]:
speeds_crashes.head()

In [ ]:
speeds_crashes.shape

In [ ]:
speeds_crashes['Lon'].value_counts()

In [ ]:
speeds_crashes['Lat'].value_counts()

In [ ]:
speeds_crashes.to_csv('speeds_crashes.csv', index=False)